In [ ]:
import numpy as np
import open3d as o3d
import carla
import time
import matplotlib.pyplot as plt
from queue import Queue
from queue import Empty
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)


client.load_world('Lidar_Testing_Ground')

In [ ]:
def draw_axes(world, location= carla.Location(0,-150,0.5), axis_length=2.0, duration=0):
    debug = world.debug
    # X axis in red
    debug.draw_line(location, location + carla.Location(x=axis_length), thickness=0.1, color=carla.Color(255, 0, 0), life_time=duration)
    # Y axis in green
    debug.draw_line(location, location + carla.Location(y=axis_length), thickness=0.1, color=carla.Color(0, 255, 0), life_time=duration)
    # Z axis in blue
    debug.draw_line(location, location + carla.Location(z=axis_length), thickness=0.1, color=carla.Color(0, 0, 255), life_time=duration)


def spawn_lidar(world, blueprint_library, location):
    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
    lidar_bp.set_attribute('horizontal_fov', '360')
    lidar_bp.set_attribute('sensor_tick', '0.1')  # 10 Hz updates

    lidar_bp.set_attribute('range', '70')
    lidar_bp.set_attribute('channels', '32')
    lidar_bp.set_attribute('points_per_second', '100000')
    lidar_bp.set_attribute('rotation_frequency', '10')
    lidar_bp.set_attribute('upper_fov', '10')
    lidar_bp.set_attribute('lower_fov', '-30')
    transform = carla.Transform(location)
    return world.spawn_actor(lidar_bp, transform)

def spawn_vehicles(world, blueprint_library, lidar_location):
    vehicle_bp = blueprint_library.filter("vehicle") [0]  # Use any available vehicle model
    
    # Calculate the y-location for vehicles: 20 meters ahead of lidar
    vehicle_y = lidar_location.y + 5
    
    # Lidar is at x=0; vehicles will be side-by-side with a 10mm offset between them
    offset = 1  # Half of 1m in meters
    vehicle1_location = carla.Location(x=-offset, y=vehicle_y, z=lidar_location.z)
    vehicle1_rotation = carla.Rotation(pitch = 0, yaw = 90, roll = 0)
    vehicle2_location = carla.Location(x=offset, y=vehicle_y, z=lidar_location.z)
    vehicle2_rotation = carla.Rotation(pitch = 0, yaw = 90, roll = 0)

    transform1 = carla.Transform(vehicle1_location, vehicle1_rotation)
    transform2 = carla.Transform(vehicle2_location, vehicle2_rotation)

    vehicle1 = world.spawn_actor(vehicle_bp, transform1)
    vehicle2 = world.spawn_actor(vehicle_bp, transform2)

    return vehicle1, vehicle2

def lidar_callback(point_cloud, sensor_queue, save_path="lidar_pointcloud.pcd"):
    # Convert raw data to numpy array of shape (N, 3)
    points = np.frombuffer(point_cloud.raw_data, dtype=np.float32)
    points = points.reshape(-1, 4)  # x, y, z, intensity
    
    # Discard intensity for visualization, keep only xyz
    xyz_points = points[:, :3]

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz_points)

    # Save point cloud to PCD file
    o3d.io.write_point_cloud(save_path, pcd)
    sensor_queue.put(point_cloud)
    print(f"Saved point cloud to {save_path}")


def visualize_pcd(pcd_path="lidar_pointcloud.pcd"):
    
    pcd = o3d.io.read_point_cloud(pcd_path) # Load the point cloud
    points = np.asarray(pcd.points) # Get numpy array of point
    distances = np.linalg.norm(points, axis=1) # Compute distance from origin for each point
    distances_normalized = (distances - distances.min()) / (distances.max() - distances.min() + 1e-8) # Normalize distances between 0 and 1
    cmap = plt.get_cmap("viridis") # Use matplotlib colormap (e.g., viridis) to map distances to colors
    colors = cmap(distances_normalized)[:, :3]  # Extract RGB, ignore alpha
    
    # Assign colors to point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Create visualizer
    vis = o3d.visualization.Visualizer() 
    vis.create_window()
    
    # Set background color to black
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()

In [1]:
import numpy as np
import open3d as o3d
import carla
import time
import matplotlib.pyplot as plt
from queue import Queue, Empty

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
client.load_world('Lidar_Testing_Ground')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:

# Draws coordinate axes (RGB: XYZ) in the CARLA world for reference
def draw_axes(world, location=carla.Location(0, -150, 0.5), axis_length=2.0, duration=0):
    debug = world.debug
    debug.draw_line(location, location + carla.Location(x=axis_length), 0.1, carla.Color(255, 0, 0), duration)  # X-axis
    debug.draw_line(location, location + carla.Location(y=axis_length), 0.1, carla.Color(0, 255, 0), duration)  # Y-axis
    debug.draw_line(location, location + carla.Location(z=axis_length), 0.1, carla.Color(0, 0, 255), duration)  # Z-axis

# Spawns a LiDAR sensor at the specified location
def spawn_lidar(world, blueprint_library, location):
    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
    # Configure LiDAR attributes
    lidar_bp.set_attribute('horizontal_fov', '360')
    lidar_bp.set_attribute('sensor_tick', '0.1')
    lidar_bp.set_attribute('range', '50')
    lidar_bp.set_attribute('channels', '64')
    lidar_bp.set_attribute('points_per_second', '100000')
    lidar_bp.set_attribute('rotation_frequency', '10')
    lidar_bp.set_attribute('upper_fov', '10')
    lidar_bp.set_attribute('lower_fov', '-30')
    transform = carla.Transform(location)
    return world.spawn_actor(lidar_bp, transform)

# Spawns two vehicles in front of the LiDAR sensor
def spawn_vehicles(world, blueprint_library, lidar_location):
    vehicle_bp = blueprint_library.filter("vehicle")[0]
    vehicle_y = lidar_location.y + 5
    offset = 2.0  # Vehicles spaced apart by 2m
    vehicle1_location = carla.Location(x=-offset, y=vehicle_y, z=lidar_location.z)
    vehicle2_location = carla.Location(x=offset, y=vehicle_y, z=lidar_location.z)
    rotation = carla.Rotation(pitch=0, yaw=90, roll=0)

    vehicle1 = world.spawn_actor(vehicle_bp, carla.Transform(vehicle1_location, rotation))
    vehicle2 = world.spawn_actor(vehicle_bp, carla.Transform(vehicle2_location, rotation))
    return vehicle1, vehicle2

# Callback function that processes LiDAR data and stores it as a PCD file
def lidar_callback(point_cloud, sensor_queue, save_path="lidar_pointcloud.pcd"):
    points = np.frombuffer(point_cloud.raw_data, dtype=np.float32).reshape(-1, 4)
    xyz_points = points[:, :3]  # Ignore intensity
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz_points)
    o3d.io.write_point_cloud(save_path, pcd)
    sensor_queue.put(point_cloud)
    print(f"Saved point cloud to {save_path}")

# Visualizes the saved point cloud using Open3D
def visualize_pcd(pcd_path="lidar_pointcloud.pcd"):
    pcd = o3d.io.read_point_cloud(pcd_path)
    points = np.asarray(pcd.points)
    if len(points) == 0:
        print("No points found in point cloud.")
        return

    # Color points based on distance
    distances = np.linalg.norm(points, axis=1)
    distances_normalized = (distances - distances.min()) / (distances.max() - distances.min() + 1e-8)
    cmap = plt.get_cmap("viridis")
    colors = cmap(distances_normalized)[:, :3]
    pcd.colors = o3d.utility.Vector3dVector(colors)

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()

# Main simulation setup and data collection logic
def main():
    world = client.get_world()
    blueprint_library = world.get_blueprint_library()

    original_settings = world.get_settings()
    lidar = None
    vehicle1 = vehicle2 = None

    try:
        # Enable synchronous mode for deterministic simulation
        settings = carla.WorldSettings(
            no_rendering_mode=False,
            synchronous_mode=True,
            fixed_delta_seconds=0.1)
        world.apply_settings(settings)

        sensor_queue = Queue()
        lidar_location = carla.Location(x=0, y=-150, z=1.2)

        # Spawn LiDAR and vehicles
        lidar = spawn_lidar(world, blueprint_library, lidar_location)
        lidar.listen(lambda data: lidar_callback(data, sensor_queue, save_path="single_frame.pcd"))

        vehicle1, vehicle2 = spawn_vehicles(world, blueprint_library, lidar_location)

        print("Collecting a single LiDAR frame...")
        world.tick()
        try:
            _ = sensor_queue.get(timeout=2.0)
            print("LiDAR frame collected and saved.")
        except Empty:
            print("Warning: No data from LiDAR sensor")

    finally:
        print("Cleaning up...")
        if lidar:
            lidar.stop()
            lidar.destroy()
        if vehicle1:
            vehicle1.destroy()
        if vehicle2:
            vehicle2.destroy()
        world.apply_settings(original_settings)

    # Visualize the collected point cloud
    visualize_pcd("single_frame.pcd")

